#LiveCon Printer
This is a proof of concept, to directly producting a Program book,
out of the LiveCon CSV.


In [1]:
import pylatex.base_classes.command
def patched_commandbase_eq(self,other):
    if isinstance(other, CommandBase):
        return self._CommandBase__key() == other._CommandBase__key()
    return False
pylatex.base_classes.command.CommandBase.__eq__=patched_commandbase_eq

In [2]:
from sessions import *

In [3]:
from pylatex.base_classes import *
from pylatex.base_classes.command import Command,Arguments


class TcboxFit(Command):  
    def __init__(self,arguments=None,options=None):
        packages = [Package('tcolorbox', options='fitting')]
        Command.__init__(self, "tcboxfit", arguments=arguments, options=options, packages=packages)

class FixedTextbox(LatexObject): #Hack: THis should be a container
    def __init__(self,text, hpos,vpos, height,width, tcb_options=None):
        self.text=text
        self.hpos=hpos
        self.vpos=vpos
        self.height=height
        self.width=width
        
        self.tcboxfit = TcboxFit(options=tcb_options, arguments=text)
        self.tcboxfit.options._key_value_args["height"]=height
        
        self.packages = [Package('textpos', options='absolute'),Package('tcolorbox', options='fitting')]
        LatexObject.__init__(self)
    
    def dumps(self): 
        return """
\\begin{textblock*}{%(width)s}(%(hpos)s, %(vpos)s)
    %(tcbox)s
\\end{textblock*}
        """ % {'width':self.width, 'hpos':self.hpos, 'vpos':self.vpos, 'tcbox':self.tcboxfit.dumps()}
    
class TextcolorboxStyle(LatexObject):
    def __init__(self, *args, **kwargs):
        self.args = args
        self.kwargs= kwargs
        self.packages = [Package('tcolorbox', options='fitting')]
        LatexObject.__init__(self)

    def dumps(self): 
        def express_arguments(args,kwargs):
            args_strs = list(map(str,args))
            kwargs_strs = ["%s=%s" % (k,v) for (k,v) in kwargs.items()]
            return ",".join(args_strs + kwargs_strs)
        return "\\tcbset{%s}" % express_arguments(self.args, self.kwargs)

def textpos_origin(x,y):
    return Command('textblockorigin',arguments=[x,y])

In [4]:
def get_tag_colors(sessions):
        from collections import defaultdict
        tag_occurances = defaultdict(lambda: 0)
        for session in sessions:
            for tag in session.tags:
                tag_occurances[tag]+=1

        tags = list(tag_occurances)
        tags.sort(key=lambda tt: -tag_occurances[tt])

        colors=["blue","brown","cyan","green","lime","magenta","olive","orange","pink","purple","red","teal","violet", "yellow"]
        color_map = {tag:color for tag,color in zip(tags,colors)}
        return color_map

#Overwrite
def get_tag_colors(sessions):
    return {' Needs Moderator': 'green',
    ' Needs Panelists': 'red'}
    

In [5]:
def fixed_venue_order(sessions):
    return [
        "South Ballroom (Large Panel Room)",
        "North Ballroom (Large Panel Room)",
        "Hamersley North (Small Panel Room)",
        "Hamersley South (Discussion Room)",
        "Boardroom (Family Room)",
        "Centre Ballroom (Traders and Art Show)",
        "Pilbara (Video Gaming Room)",
        "Goldsworthy (Gaming Room)"]

In [6]:
def with_units(func):
    def wrapper(*args, **kwargs):
        units = args[0].units #Assumes that this is decorating a method on a class with a units field
        result = func(*args, **kwargs)
        return str(round(result,3))+units
    return wrapper
    
class timetable_metric_solver():
    import venueordering
    #Hack Me to have specified order of venues
    def __init__(self,sessions, hour_len, venue_width, units, 
                 voffset=0,
                 venue_orderer = venueordering.get_order_from_sessions):
        
        
        self.voffset = voffset
        self.hour_len = hour_len
        self.venue_width = venue_width
        self.units = units
        
        self.venues_to_x = { name:ii*venue_width
                                for ii, name in enumerate(venue_orderer(sessions))}
        
        from itertools import groupby
        self.day_starts = {date:min(map(lambda ss: ss.start, sessions))
                                for date, sessions in groupby(sessions, lambda ss: ss.start.date())}
        self.tag_colors = get_tag_colors(sessions)
    
    def get_color(self,session):
        if session.tags:
            mix = round(95/len(session.tags)) #paleness
            colors = [self.tag_colors[tag]+ "!%s!" % mix 
                          for tag in session.tags if tag in self.tag_colors]
            if colors:
                return ''.join(colors)+"white"
            else:
                return ""
        else:
            return ""
        
    
    @with_units
    def get_width(self,session):
        if session.venues:
            return len(session.venues)*self.venue_width    
        else:
            nVenues = len(self.venues_to_x.keys())
            return nVenues*self.venue_width    
    
    @with_units
    def get_venue_x(self,venue):
        return self.venues_to_x[venue]
    
    @with_units
    def get_x(self, session):
        #Get the left most venue position
        if session.venues:
            return min(map(lambda vv: self.venues_to_x[vv],session.venues))
        else:
            return 0 # fills all Venues, starting from left
            
    
    def duration_to_height(self,duration):
        return duration.seconds/(60*60)*self.hour_len
    
    @with_units
    def get_y(self, session):
        day_start = self.day_starts[session.start.date()]
        time_til_start = session.start - day_start
        return self.voffset + self.duration_to_height(time_til_start)
    
    @with_units
    def get_height(self,session):
        return self.duration_to_height(session.duration)
    
    @with_units
    def get_venue_width(self):
        return self.venue_width
    
    @property
    def venues(self):
        return list(self.venues_to_x.keys())
    
    

In [40]:
from pylatex.utils import escape_latex, NoEscape
from pylatex.utils import NoEscape
from pagelayout import Multicols
from itertools import groupby

def write_descriptions(sessions, doc):
     with doc.create(Multicols(2)):
        for day_name, day_session in groupby(sessions, lambda ss: ss.day):
            with doc.create(Section(day_name,numbering=False)):
                for session in day_session:
                    if len(session.description)==0:
                        continue

                    with doc.create(Subsection(session.title,numbering=False)):
                        with doc.create(Description()) as desc:
                            doc.append(Command("setlength",[NoEscape("\itemsep"),"0pt"]))
                            doc.append(Command("setlength",[NoEscape("\parsep"),"0pt"]))
                            doc.append(Command("setlength",[NoEscape("\parskip"),"0pt"]))
                            desc.add_item("When:", session.day+", "+session.start_time+" -- "+session.end_time)

                            if len(session.venues)>0:
                                desc.add_item("Where:", ", ".join(session.venues))
                            if len(session.people)>0:
                                desc.add_item("Who:", ", ".join(session.people))
                            #if len(session.tags)>0:
                            #    desc.add_item("Tags:", ", ".join(session.tags))
                        doc.append(session.description)

In [41]:
from pylatex.utils import escape_latex, NoEscape
from itertools import groupby
from pylatex.base_classes.command import Options
from pylatex.utils import escape_latex

def write_venues(doc, tt_solver):
    doc.append(textpos_origin('2cm','2cm'))
    doc.append(TextcolorboxStyle('sharp corners','fit algorithm=hybrid*','center upper', valign='center',
                                 colframe='blue!50!black',colback='blue!10!white',
                                 boxsep='0pt',top='0mm',bottom='0mm',left='0mm',right='0mm'))
            
    for venue in tt_solver.venues:
        doc.append(FixedTextbox(venue,
                                tt_solver.get_venue_x(venue),
                                '0cm', 
                                '1.5cm',
                                tt_solver.get_venue_width())
                               )

def make_pretty_timetable(doc,sessions):
    tt_solver = timetable_metric_solver(sessions,1.7,2.15,'cm', 1.7, venue_orderer=fixed_venue_order)

    
    for date, day_sessions in groupby(sessions, lambda ss: ss.start.date()):   
        day_str = str(DAYS[date.weekday()])
        with doc.create(Subsection(NoEscape(day_str+" \hfill "+day_str+" \hfill "+day_str), numbering=False)):
           
            write_venues(doc,tt_solver)
            doc.append(TextcolorboxStyle('fit algorithm=hybrid*','rounded corners', 'center upper', valign='center',
                                 colframe='blue!50!black',colback='white!10!white',
                                 boxsep='1pt',top='0mm',bottom='0mm',left='0mm',right='0mm'))
            
            day_sessions = sorted(day_sessions, key = lambda ss: -len(ss.venues) or -len(tt_solver.venues)-1)
            for session in day_sessions:
                print("*", session.title)
                colback = tt_solver.get_color(session)
                tcb_options = Options(colback=colback) if colback else None
                doc.append(FixedTextbox(NoEscape('%s \\\\ \\tcbfontsize{0.75} %s -- %s ' % 
                                                     tuple(map(escape_latex, (session.title, session.start_time, session.end_time)))),
                                        tt_solver.get_x(session),
                                        tt_solver.get_y(session), 
                                        tt_solver.get_height(session),
                                        tt_solver.get_width(session),
                                        tcb_options=tcb_options))

            doc.append(Command('newpage'))



In [42]:
import csv

sessions = []
sessions = load_sessions_csv('con_data/sc2016.csv')
sessions = list(filter(lambda ss: not "NotInPrintProgram" in ss.tags, sessions))
sessions.sort(key=lambda ss: ss.start)


#sessions = list(filter(lambda ss: ss.day == "Monday", sessions))
#sessions = list(filter(lambda ss: 'Jane Espenson' in ss.people, sessions))
#sessions = list(filter(lambda ss: 'Gillian Rubinstein' in ss.people, sessions))
#sessions = list(filter(lambda ss: 'Fe Waters' in ss.people, sessions))
#sessions = list(filter(lambda ss: "Michael O'Brien" in ss.people, sessions))
#sessions = list(filter(lambda ss: " Gaming" in ss.tags or "Gaming" in ss.tags, sessions))
#sessions = list(filter(lambda ss: "Family" in ss.tags or " Family" in ss.tags, sessions))
len(sessions)

193

In [43]:
import pylatex
from pylatex import Document, Section, Subsection, Subsubsection, Table, Package, lists
from pylatex.lists import Description

from tabulartimetable import write_timetable


doc = Document(documentclass="scrreprt")
margins=['tmargin=1.5cm','bmargin=2.5cm','lmargin=2.5cm','rmargin=2.5cm',]
doc.packages.append(Package('geometry', options=margins))
doc.packages.append(Package("draftwatermark"))
doc.append(Command("newgeometry",arguments=",".join(margins)))
doc.append(Command("SetWatermarkText", "Draft 6"))
doc.append(Command("SetWatermarkScale", "0.5"))
doc.append(Command("SetWatermarkColor", "0.9,0.59,0.5", "rgb"))

make_pretty_timetable(doc,sessions)
write_descriptions(sessions,doc)

from IPython.display import FileLink, FileLinks
with open("out/exported.tex", 'w') as fh:
    doc.dump(fh)

!lualatex --output-directory=out --interaction=nonstopmode out/exported.tex
FileLinks("./out")
FileLinks("./out")


* Dinner
* Opening Ceremony
* Open console gaming
* Meet the AV Guy
* Hitch Hikers Guide to Swancon
* Z-Z-Z-Zenith
* Safe Spaces
* Moderating the Moderators
* Why You should run Swancon 2018
* Crime Fighting Flies
* Ch-ch-ch-changeling - The Many Faces of Bowie
* Fantasy that is not Western European
* Let's talk about boobs, baby.
* What does a Day at Work look like for me?
* Undead dilemmas
* Attracting and Keeping New Members
* New Epics: Deeper Plots, Greater Magics
* Beefcake The Fourth!
* Adult Themes Trailer Park
* Lunch
* Dinner, (including the Pizza Restaurant at the End of the Universe)
* There is no shame in enjoying Paranormal Romance
* How to Sort your Library
* Collaborative Storytelling Games
* Martial Arts and Martial Characters in SpecFic
* Mash up Comics
* Hyrule warriors
* What makes a Good Game Start?
* How can we help to get more younger children to read, what are the strategies?
* So you Want to Make a Video Game
* The Best Book I read in 2015 was
* Frankenplushies

./out/
  exported.pdf
  exported.log
  exported.tex
  exported.aux

In [ ]:
#ROOMs/DAYS

import pylatex
from pylatex import Document, Section, Subsection, Subsubsection, Table, Package
from pylatex.command import Command
from lists import Description

from itertools import groupby

from collections import defaultdict
dvs = defaultdict(lambda : defaultdict(list))
for date, day_sessions in groupby(sessions, lambda ss: ss.start.date()):
    for sess in day_sessions:
        for venue in sess.venues:
            dvs[date][venue].append(sess)

#################################
doc = Document(documentclass="scrreprt")
#doc.packages.add("enumitem")
#doc.append(r"\setlist[description]{leftmargin=10em,labelindent=0em")
doc.append(r"\setkomafont{section}{\LARGE}")
for date in dvs.keys():
    day = DAYS[date.weekday()]

    for venue in dvs[date]:
        doc.append("\\thispagestyle{empty}")
        with doc.create(Section("%s -- %s" % (day,venue), numbering=False)):
            with doc.create(Description()) as sched:
                doc.append("\\Large")
                for sess in sorted(dvs[date][venue], key=lambda ss: ss.start):
                    sched.additem("%s -- %s" % (sess.start_time, sess.end_time), sess.title)
            doc.append("\\newpage")
                

In [ ]:
from IPython.display import FileLink, FileLinks
with open("out/temp_days.tex", 'w') as temp_out:
    doc.dump(temp_out)

#!lualatex --output-directory=out --interaction=nonstopmode temp_days.tex
FileLinks("./out")
